In [5]:
# importing all necessary libs
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objs as go
import dash
import os
import dash_core_components as dcc
import dash_html_components as html
from plotly.subplots import make_subplots
import py7zr
import warnings
warnings.filterwarnings("ignore")

import scipy.stats
from scipy.stats import expon


/var/folders/26/9mt94nq12r58hxyl1vm__cs00000gn/T/ipykernel_29997/3903838457.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/26/9mt94nq12r58hxyl1vm__cs00000gn/T/ipykernel_29997/3903838457.py:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [7]:
curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)

# Data reading
with py7zr.SevenZipFile(parent_dir + '/data/globalterrorismdb_0221dist.7z') as z:
    # open the csv file in the dataset
    targetPath = os.path.expanduser("~/Desktop/GT Dataset")
    z.extract(path = targetPath)
    df = pd.read_excel(targetPath + '/globalterrorismdb_0221dist.xlsx')
    
# We filtered out doubted attacks to be able to have exact attacks
df = df[df['doubtterr'] != 1]

_df = df[['eventid','iyear','imonth','iday','country','country_txt',
         'region','region_txt', 'provstate', 'city','latitude','longitude',
         'gname','nkill', 'nwound']]

In [8]:
# Geojson for polygons of map
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
    countries = json.load(response)
                          
countries_df = pd.json_normalize(countries,  record_path =['features'])

In [9]:
# Updating the countries to be able to merge -- TODO: !!!!
_df.loc[_df['country_txt'] == 'Bahamas','country_txt']  = 'The Bahamas'
_df.loc[_df['country_txt'] == 'United States','country_txt'] = 'United States of America'
_df.loc[_df['country_txt'] == 'Czechoslovakia','country_txt'] = 'Czech Republic'
_df.loc[_df['country_txt'] == 'East Germany (GDR)','country_txt']  = 'Germany'
_df.loc[_df['country_txt'] == 'West Germany (FRG)','country_txt']  = 'Germany'

_df.loc[_df['country_txt'] == 'West Bank and Gaza Strip','country_txt'] = 'West Bank'
_df.loc[_df['country_txt'] == 'South Vietnam','country_txt'] = 'Vietnam'
_df.loc[_df['country_txt'] == 'North Yemen','country_txt'] = 'Yemen'
_df.loc[_df['country_txt'] == 'South Yemen','country_txt'] = 'Yemen'
#_df.loc[_df['country_txt'] == 'Andorra','country_txt'] = 'Spain'
_df.loc[_df['country_txt'] == 'Bahrain','country_txt'] = 'Iran'
_df.loc[_df['country_txt'] == 'Tanzania','country_txt'] = 'United Republic of Tanzania'

for ind in _df.index:
    if _df.loc[ind,'country_txt'] in ['Soviet Union', 'Yugoslavia']:
        _df.loc[ind,'country_txt'] = _df.loc[ind,'provstate']


_df.loc[_df['country_txt'] == 'Bosnia-Herzegovina','country_txt'] = 'Bosnia and Herzegovina'
_df.loc[_df['country_txt'].isin(['Central Serbia','Serbia','Serbia-Montenegro',
                                 'Republika Srpska','Belgrade','Kosovo and Metohija']),'country_txt'] = 'Republic of Serbia'  
_df.loc[_df['country_txt'] == 'Yugoslavia','country_txt'] = 'Republic of Serbia'
_df.loc[_df['country_txt'] == 'International','country_txt'] = 'United Arab Emirates'
_df.loc[_df['country_txt'] == 'Hong Kong','country_txt'] = 'China' 
_df.loc[_df['country_txt'] == 'Kygyzstan','country_txt'] = 'Kyrgyzstan'
_df.loc[_df['country_txt'] == 'Maldives','country_txt'] = 'India'

In [10]:
# Merging locations and main df
df_merged = pd.merge(_df, countries_df , left_on = 'country_txt', right_on = 'properties.name', how = 'left' )

# Determine ig organized or unorganized
df_merged['isOrganized'] = ['unorganized' if i == 'Unknown' else 'organized' for i in df_merged['gname']]



In [11]:
def grouped_weighted_avg(values, weights, by):
    return (values * weights).groupby(by).sum()  / weights.groupby(by).sum()

In [12]:
weights =  np.linspace(expon.ppf(0.01), expon.ppf(0.99), len(df_merged.iyear.unique()))
weights = weights/ np.sum(weights)

years = df_merged.iyear.unique()
weighted_years = pd.DataFrame({'years': years, 'weights': weights})

_df = pd.merge(df_merged, weighted_years, left_on = 'iyear', right_on = 'years', how = 'inner')
_df['total_kills_injured'] = _df['nkill'] + _df['nwound']



In [ ]:
# We need safety index calculation before groupby
def update_map():
    df_map = _df.groupby(by=['country','country_txt','id', 'iyear']).agg({
                                                                'nkill': np.sum,
                                                                'nwound': np.sum,
                                                                'total_kills_injured':np.sum,
                                                                'eventid': 'count', 
                                                                'weights': np.mean}).reset_index()  #weights same for same years
    
    calculated_indexes =  pd.DataFrame(grouped_weighted_avg(values=df_map.eventid, 
                                                    weights=df_map.weights, 
                                                    by=df_map.id)).reset_index().rename(columns={0: 'calculated_index'})
    
    df_map = df_map.groupby(by=['country','country_txt','id']).agg({
                                                                'nkill': np.sum,
                                                                'nwound': np.sum,
                                                                'total_kills_injured':np.sum}).reset_index()
    
    df_map = pd.merge(df_map, calculated_indexes, on = 'id', how = 'left')

    df_map['hover_text'] = 'Country: ' + df_map["country_txt"].astype(str) + "<br>" + 'Unsafety Index: ' + df_map['calculated_index'].round(decimals= 2).astype(str) + '<br>'+'# of Killed and Wounded People: ' +  df_map['total_kills_injured'].astype(str)+ '<br>'+'# of Killed People: '+  df_map['nkill'].astype(str)+ '<br>'+ '# of Wounded People: '+  df_map['nwound'].astype(str)
    
    trace = go.Choropleth(locations=df_map['id'],
                          z=df_map['calculated_index'],
                          text=df_map['hover_text'], 
                          hoverinfo='text' ,
                          colorscale="rdylgn",
                          reversescale = True,
                          marker={'line': {'color': 'rgb(180,180,180)','width': 0.5}},
                          colorbar={"thickness": 20,"len": 0.7, "x": 0.9, "y": 0.7,
                                   'title': {"text": 'Safety Index', "side": "bottom"},
                                 }
                         )   
    return {"data": [trace],
            "layout": go.Layout(height=800, width= 1200 ,
                                geo={'showframe': False,
                                     'showcoastlines': False,
                                     'projection': {'type': "miller"}})}



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = 'Dashboard'


app.layout = html.Div([
    html.Div([html.H1("Global Terrorism Dashboard")],
                 style={'textAlign': "center", "padding-bottom": "30"}, 
                 className="six-column"),
    html.Div([dcc.Graph(id="world-map", 
                       style = {'textAlign': "center"}, 
                       figure = update_map(), className="twelve columns"), 
              html.A('Source: Global Terrorism Database(GTD)',href = 'https://www.start.umd.edu/gtd/', 
                      style={'fontStyle': "italic", 'color': '#273746'})]), 
    html.Div([html.H2('First 15 Countries with Most Organized and Unorganized Attacks'),
              dcc.Graph(id="horizontal-bar")], 
             className="twelve columns"),
    html.H2(id = 'country-name',  children=["init"] , 
            style={'textAlign': "center", "padding-bottom": "30"} ),
    html.Div(dcc.Graph(id="incidents"),  id = 'incident-box', 
             style = {'visibility': 'hidden'}, className="five columns"),
    html.Div(dcc.Graph(id="fatalities"),  id = 'fatalities-box', 
             style = {'visibility': 'hidden'}, className="five columns"),
    
])


##### Figure Updates with CallBacks ##### 
@app.callback(
    dash.dependencies.Output('country-name', 'children'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_title(clickData):
    text = ''
    if clickData is not None:
        selected_country = clickData['points'][0]['location']
        text = str(selected_country)  
    return text

@app.callback(
    dash.dependencies.Output('horizontal-bar', 'figure'),
    [dash.dependencies.Input('world-map', 'clickData')])
def update_horizontal_bar(clickData):
    def get_data(year):
        df_h = df_merged[df_merged['iyear'] == year].groupby(by=['country','country_txt',
                                       'id', 'isOrganized']).agg({'eventid':'count'}).unstack(fill_value=0).stack().sort_values(by='country').reset_index()
        df_x = df_h.pivot_table(index=['country','country_txt'], columns='isOrganized')
        countries = df_h.drop_duplicates(subset=['country','country_txt']).reset_index()[['country','country_txt']]
        countries['organized'] = df_x['eventid']['organized'].reset_index()['organized']
        countries['unorganized'] = df_x['eventid']['unorganized'].reset_index()['unorganized']
        countries['Total'] = countries['organized'] +countries['unorganized']
        countries = countries.sort_values(by='Total', ascending= False)[0:15]

        return countries

    fig = go.Figure()
    years = df_merged['iyear'].unique()

    for step in years:
        countries = get_data(step)
        fig.add_trace(go.Bar(
            visible=False,
            y=countries['country_txt'],
            x=countries['organized'],
            name='Organized',
            orientation='h',
            marker=dict(
                color='rgba(246, 78, 139, 0.6)',
                line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
            )
        ))
        fig.add_trace(go.Bar(
            visible=False,
            y=countries['country_txt'],
            x=countries['unorganized'],
            name='Unorganized',
            orientation='h',
            marker=dict(
                color='#5885AF',
                opacity = 0.8,
                line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
            )
        ))

    fig.data[-1].visible = True
    fig.data[-2].visible = True

    steps = []
    for i in range(len(years)):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)},
                  {"title": "Top 15 most attacked countries in " + str(years[i])}],  # layout attribute
        )
        step["args"][0]["visible"][i * 2] = True  # Toggle i'th trace to "visible"
        if i * 2 < len(fig.data) - 1:
            step["args"][0]["visible"][i * 2 + 1] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=48,
        currentvalue={"prefix": "Selected Year: "},
        pad={"t": 50},
        steps=steps
    )]

    fig.update_layout(
        sliders=sliders,
        barmode='stack'
    )

    fig['layout']['sliders'][0]['currentvalue']['prefix']='Year: '
    for i, date in enumerate(years, start = 0):
        fig['layout']['sliders'][0]['steps'][i]['label']=str(date)
    return fig
        



@app.callback(
    [dash.dependencies.Output('incidents', 'figure'), dash.dependencies.Output('incident-box', 'style')],
    [dash.dependencies.Input('world-map', 'clickData')])
def update_incidents(clickData):
    df_agg = ''
    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df_agg = df_merged[df_merged['id'] == selected_country].groupby(by=['iyear','country',
                                                                            'country_txt','id', 
                                                                            'isOrganized']).agg({
                                                                                        'eventid':'count'}).reset_index()
    fig = px.line(df_agg, x="iyear", y="eventid", color='isOrganized',
                  labels={
                     "iyear": "<b>Year</b>",
                     "eventid": "<b>Number of Incidents</b>",
                     "isOrganized": " "
                 }, title="<b>Number of Incidents by Years</b>")
    
    return fig,  {'visibility':'visible'}

@app.callback(
    [dash.dependencies.Output('fatalities', 'figure'), dash.dependencies.Output('fatalities-box', 'style')],
    [dash.dependencies.Input('world-map', 'clickData')])
def update_fatalities(clickData):
    df_agg = ''
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if clickData is not None:
        selected_country =clickData['points'][0]['location']
        df_agg = df_merged[df_merged['id'] == selected_country].groupby(by=['iyear','country',
                                                                        'country_txt','id']).agg({
                                                                                        'nkill':np.sum,
                                                                                        'nwound':np.sum  }).reset_index()
    # Add traces
    fig.add_trace(
        go.Line(x=df_agg['iyear'], y=df_agg['nwound'], name="Injuries", line=dict(color="#D4AC0D")),
        secondary_y=False,
    )

    fig.add_trace(
        go.Line(x=df_agg['iyear'], y=df_agg['nkill'], name="Fatalities", line=dict(color="#943126")),
        secondary_y=True,
    )     
    
    # Add figure title
    fig.update_layout(
        title_text="<b>Number of Injuries and Fatalities by Years</b>"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>Year</b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Injuries</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Fatalities</b> ", secondary_y=True)

    
    return fig, {'visibility':'visible'}



    
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jan/2022 19:05:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/opt/homebrew/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/opt/homebrew/lib/python3.9/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "/var/fol

127.0.0.1 - - [14/Jan/2022 19:05:32] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/homebrew/lib/python3.9/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/opt/homebrew/lib/python3.9/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/opt/homebrew/lib/python3.9/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_args, **func_kwargs)  # %% callback invoked %%
  File "/var/fol

127.0.0.1 - - [14/Jan/2022 19:05:32] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:53] "POST /_dash-update-component HTTP/1.1" 200 -
/opt/homebrew/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


127.0.0.1 - - [14/Jan/2022 19:05:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2022 19:05:53] "POST /_dash-update-component HTTP/1.1" 200 -


In [67]:
def get_data(year):
    df_h = df_merged[df_merged['iyear'] == year].groupby(by=['country','country_txt',
                                   'id', 'isOrganized']).agg({'eventid':'count'}).unstack(fill_value=0).stack().sort_values(by='country').reset_index()
    df_x = df_h.pivot_table(index=['country','country_txt'], columns='isOrganized')
    countries = df_h.drop_duplicates(subset=['country','country_txt']).reset_index()[['country','country_txt']]
    countries['organized'] = df_x['eventid']['organized'].reset_index()['organized']
    countries['unorganized'] = df_x['eventid']['unorganized'].reset_index()['unorganized']
    countries['Total'] = countries['organized'] +countries['unorganized']
    countries = countries.sort_values(by='Total', ascending= False)[0:15]
    
    return countries

fig = go.Figure()
years = df_merged['iyear'].unique()
# lyear 70-19
for step in years:
    countries = get_data(step)
    fig.add_trace(go.Bar(
        visible=False,
        y=countries['country_txt'],
        x=countries['organized'],
        name='Organized',
        orientation='h',
        marker=dict(
            color='rgba(246, 78, 139, 0.6)',
            line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
        )
    ))
    fig.add_trace(go.Bar(
        visible=False,
        y=countries['country_txt'],
        x=countries['unorganized'],
        name='Unorganized',
        orientation='h',
        marker=dict(
            color='#5885AF',
            opacity = 0.8,
            line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
        )
    ))
    
fig.data[-1].visible = True
fig.data[-2].visible = True

steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Top 15 most attacked countries in " + str(years[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i * 2] = True  # Toggle i'th trace to "visible"
    if i * 2 < len(fig.data) - 1:
        step["args"][0]["visible"][i * 2 + 1] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=48,
    currentvalue={"prefix": "Selected Year: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    barmode='stack'
)

fig['layout']['sliders'][0]['currentvalue']['prefix']='Year: '
for i, date in enumerate(years, start = 0):
    fig['layout']['sliders'][0]['steps'][i]['label']=str(date)

fig.show()
